# On Dispatching Design Patters - Based on [this talk](https://www.youtube.com/watch?v=n-E-1-A_rZM).

In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/MEGA/EMAp/Julia_Tutorials/Dispatch-Design`


### The Basics

The multiple-dispatch capability of Julia makes design patterns in Julia quite different. First of all, Julia
does not have classes. Instead, it has Structs, which can be thought of as composite "types", on which
functions can be written in order to transform such struct. Hence, a struct is not like a class where one defines methods,
but it's more like an "pure" object with properties, and to which functions are applied to.

Again, while in OOP a class will generate objects that have "methods" inside of it. In Julia, there are no methods, it's all functions.
And these functions are separated from the structs.
The way we tie them together is via multiple dispatch.

An example might make things clearer.

Think of the family of geometric shapes.
Each object will be a shape, such as a square, a rectangle, a triangle, and so on. While the functions
will take shapes and perform some action.


In [2]:
struct Point2D
    x::Real
    y::Real
end

p = Point2D(1, 1);

The structs in Julia are by default immutable. This allows for more optimized code. Yet, if one needs,
the struct can be made mutable, by simply writing `mutable struct`.
Note also that whenever we define a struct, it already comes with a constructor function (e.g. `Point2D()`).
But this can be modified, as we are going to show shortly.

We can access the properties of a struct with the `p.x` notation, or using `getproperty(p, :x)`.
If you wish to know what are the properties of an object, just do `propertynames(p)`.

Let's define the abstract type `Shape`. This is useful, because it works as a hierarchical type, allowing us to write functions
that work for every struct of "hypertype" `Shape`.

In [3]:
abstract type Shape end

Next, let's create some instances of shapes. 

In [4]:
mutable struct Square <: Shape
    center::Point2D
    length::Real
    Square(center, length) = length ≥ 0 ? new(center, length) : error("length should be greater or equal than 0.")
end

□ = Square(p, 10);

The `new()` is a special function that allows one to construct the object from within the struct. We need to do so 
because the struct is not yet defined, hence the need for the `new`.

In [43]:
mutable struct Rectangle <: Shape
    center::Point2D
    height::Real
    width::Real
    Rectangle(center, height, width) = height ≥ 0 && width ≥ 0 ? new(center, height, width) : error("height and width should be greater or equal than 0.")
end

height(s::Rectangle) = s.height
width(s::Rectangle)  = s.width

height(s::Square) = s.length
width(s::Square)  = s.length

area(s::Shape)    = width(s) + height(s)
sumshapes(s1::Shape, s2::Shape) = area(s1) + area(s2);

Next, let's create a list. Our list is a nested struct. To create it, we write a `mklist` function.

In [44]:
# Straight from https://github.com/ninjaaron/dispatching-design-patterns
struct Nil end
struct List{T}
    head::T
    tail::Union{List{T}, Nil}
end

mklist(array::AbstractArray{T}) where T = foldr(List{T}, array, init=Nil())

# implement the iteration protocol
Base.iterate(l::List) = iterate(l, l)
Base.iterate(::List, l::List) = l.head, l.tail
Base.iterate(::List, ::Nil) = nothing

list = mklist(1:5)
for val in list
    println(val)
end

1
2
3
4
5


## Using Parameters and Units

In many situations, we might want our structs to have default values that could be
altered when creating a new instance. The `Parameters.jl` package makes this very easy to do.


In [45]:
using Parameters
@with_kw struct Board
    pencil::String = "black"
    background::String = "white"
    boardtype::String = "whiteboard"
    size::Tuple{Real,Real} = (10,10)
end

@show board = Board()
@show chalkboard = Board(pencil="white", background="green", boardtype="chalkboard");
println("----------------------------------------------------------------------------")

board = Board() = Board
  pencil: String "black"
  background: String "white"
  boardtype: String "whiteboard"
  size: Tuple{Int64, Int64}

chalkboard = Board(pencil = "white", background = "green", boardtype = "chalkboard") = Board
  pencil: String "white"
  background: String "green"
  boardtype: String "chalkboard"
  size: Tuple{Int64, Int64}

----------------------------------------------------------------------------


The struct above is great, but it could be improved. Note for example that it's possible in this struct to
declare a board with negative size, which is nonsense. Also, the size has no units, which makes it meaningless.
We could add another field to state the units, but there is a much more elegant way using the amazing
`Unitful.jl` package.

In [48]:
using Unitful

@with_kw struct NewBoard
    pencil::String = "black"
    background::String = "white"
    boardtype::String = "whiteboard"
    size::Tuple{Unitful.Length,Unitful.Length} = (100u"cm",100u"cm")
    function NewBoard(pencil, background, boardtype, size)
        if size[1]≥ 0u"cm" && size[2] ≥ 0u"cm"
            return new(pencil, background, boardtype, size)
        else
            error("Size should be non-negative.")
        end
    end
end

@show newboard = NewBoard();
println("----------------------------------------------------------------------------")

newboard = NewBoard() = NewBoard
  pencil: String "black"
  background: String "white"
  boardtype: String "whiteboard"
  size: Tuple{Quantity{Int64, 𝐋, Unitful.FreeUnits{(cm,), 𝐋, nothing}}, Quantity{Int64, 𝐋, Unitful.FreeUnits{(cm,), 𝐋, nothing}}}

----------------------------------------------------------------------------


This is nice, but `Unitful.jl` enables us to control this in a much more secure manner.
Suppose, for example, that we wish to send this specs to an american woodworker. Hence,
we need to convert the units for our table.

In [55]:
uconvert.(u"ft",newboard.size)

(1250//381 ft, 1250//381 ft)